In [ ]:
#@title Importing Libraries
from google.colab import files
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from gtts import gTTS
from datetime import datetime
import pytz

In [ ]:
# Checking the current runtime
availableRuntime="GPU" if torch.cuda.is_available() else "CPU"
availableRuntime

'CPU'

In [ ]:
#@title Phase 1: Uploading image
# uploading image file and returning file name
def uploadImgFile():
  # Prompt the user to upload a file
  uploaded = files.upload()
  uploadedFileName=""
  # Print the uploaded file name
  for filename in uploaded.keys():
      uploadedFileName=filename
  return uploadedFileName

In [ ]:
fileName=uploadImgFile()

Saving ghost.jpeg to ghost.jpeg


In [ ]:
#@title Phase 2: Image to Text conversion
#This code snippet defines a function BlipImagetoText(fileName) that
#performs image captioning using the Blip model for both conditional and unconditional image captioning.
def BlipImagetoText(fileName):
  processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
  model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda") if availableRuntime == "GPU" else BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

  raw_image = Image.open(fileName).convert('RGB')
  # conditional image captioning
  text = "a photograph of "
  inputs = processor(raw_image, text, return_tensors="pt").to("cuda") if availableRuntime == "GPU" else processor(raw_image, text, return_tensors="pt")

  out = model.generate(**inputs)
  #print(processor.decode(out[0], skip_special_tokens=True))

  # unconditional image captioning
  inputs = processor(raw_image, return_tensors="pt").to("cuda")
  #set HF_TOKEN variable in google colab secret and get secret key from https://huggingface.co/settings/tokens
  out = model.generate(**inputs,max_new_tokens=200)
  return processor.decode(out[0], skip_special_tokens=True)

In [ ]:
textResult=BlipImagetoText(fileName)
textResult

In [ ]:
#@title Phase 3: Prompt generated text into LLM Model
from openai import OpenAI
from google.colab import userdata
#save openai access key in notebook secret by name =openAIsecretKey  and provide acess to the notebook
open_api_secret_name = 'openAIsecretKey'  # @param {type: "string"}

try:
  OPEN_API_KEY=userdata.get(open_api_secret_name)
  client=OpenAI(api_key=OPEN_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {open_api_secret_name} in Colab')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {open_api_secret_name} secret in order for the notebook to access openai on your behalf.')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {open_api_secret_name} stored in Colab.")
  raise e

In [ ]:
#@title Open AI Model Prompt Creation
model="gpt-3.5-turbo"
descriptionQuery=textResult
def openaiInference(model,descriptionQuery):
  completion = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": "you are a security observer. An image was clicked by suvelliance camer you get the {description} from the image create an announcement raising security threat if you find something life threatening , otherwise just brief the description without raising any threat alarm."
    },
    {"role": "user", "content":f"{descriptionQuery}"}
    ]
    )

  return " ".join(completion.choices[0].message.content.split("\n"))

In [ ]:
llmInference = openaiInference(model,descriptionQuery)
llmInference

In [ ]:
#@title Phase 4: Text to speech conversion
text_to_say=llmInference # text to convert into audio
language='en'# gTTS support total 63 languages, here I am using english
folderPath="/content/" # for example - "/content/"

globalfilepath=""
def audioWithFileSave(text,lang,folderPath):
  global globalfilepath
  # Get the current time in Pacific Time
  pacific_time = datetime.now(pytz.timezone('America/Los_Angeles'))
  # Format the current time as a string
  formatted_time = pacific_time.strftime("%Y-%m-%d %H:%M:%S")
  gtts_obj=gTTS(text=text,lang=lang,slow=False)
  folderPath=folderPath+"audio+"+str(formatted_time).replace(" ","_")+".mp3"
  globalfilepath=folderPath
  gtts_obj.save(folderPath)

In [ ]:
#Calling function to same audio file
audioWithFileSave(text_to_say,language,folderPath)

In [ ]:
#@title Playing audio file in google colab
from IPython.display import Audio
Audio(globalfilepath)